In [2]:
%load_ext autoreload
%autoreload 2
%cd ..

/media/minhduc0711/Libraries/Codes/EURECOM/MALIS_project


In [83]:
import mido
import numpy as np
import pandas as pd

from src.data.midi import parse_midi

In [89]:
mid_path = "data/raw/MAPS/ENSTDkAm2/ENSTDkAm/MUS/MAPS_MUS-chpn_op25_e3_ENSTDkAm.mid"
mid = mido.MidiFile(mid_path)

In [94]:
# original txt file from MAPS dataset
ref_df = pd.read_csv("data/raw/MAPS/ENSTDkAm2/ENSTDkAm/MUS/MAPS_MUS-chpn_op25_e3_ENSTDkAm.txt", delim_whitespace=True)
ref_df.sort_values(["OnsetTime", "OffsetTime"], inplace=True, ignore_index=True)
print(len(ref_df))

1692


In [95]:
# my code
df = parse_midi(mid_path)
err = np.abs(df[["onset", "offset"]].to_numpy() - ref_df[["OnsetTime", "OffsetTime"]].to_numpy())
np.sum(err, axis=0)

array([0.04010921, 0.04173317])

In [56]:
# cross-check with pytorch repo
df2 = pd.read_csv("MAPS_MUS-bk_xmas1_ENSTDkAm.tsv", delimiter="\t")
df2.sort_values(["onset", "offset"], inplace=True, ignore_index=True)
err = np.abs(df2[["onset", "offset"]].to_numpy() - ref_df[["OnsetTime", "OffsetTime"]].to_numpy())
np.sum(err, axis=0)

array([3.94023000e-01, 1.04846162e+03])

In [96]:
# cross-check with magenta
from note_seq import midi_io
seq = midi_io.midi_file_to_note_sequence(mid_path)
rows = []
for note in seq.notes:
    rows.append({
        "onset": note.start_time,
        "offset": note.end_time,
        "note": note.pitch
    })
df_magenta = pd.DataFrame(rows, columns=["onset", "offset", "note", "velocity"], )
df_magenta.sort_values(["onset", "offset"], inplace=True, ignore_index=True)
err = np.abs(df_magenta[["onset", "offset"]].to_numpy() - ref_df[["OnsetTime", "OffsetTime"]].to_numpy())
np.sum(err, axis=0)

array([4.01092102e-02, 3.59860614e+02])